In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

df = pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')
# print(df.columns)
features = ["Utilities","Neighborhood", "BldgType", "YearBuilt", "YearRemodAdd", "GrLivArea", "LotArea", "CentralAir",
            "BedroomAbvGr", "GarageCars", "OverallCond"]

# print(df[features].head())

label_encoder = LabelEncoder()
df["Utilities"] = label_encoder.fit_transform(df["Utilities"])
df["Neighborhood"] = label_encoder.fit_transform(df["Neighborhood"])
df["BldgType"] = label_encoder.fit_transform(df["BldgType"])
df["CentralAir"] = df["CentralAir"].map({'Y': 1, 'N': 0})

# print(df[features].head())
X = df[features]
y = df.SalePrice

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = RandomForestRegressor(n_estimators=500, random_state=42)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Step 1: Add a small value to prevent taking log of 0
epsilon = 1e-6
predictions = predictions + epsilon
y_test = y_test + epsilon
 
log_predictions = np.log(predictions)
log_y_test = np.log(y_test)
rmse_log = np.sqrt(mean_squared_error(log_predictions, log_y_test))
# mse = mean_squared_error(predictions, y_test)
print(rmse_log)
# random forest - 0.18224137960318498
# gradient boost - 0.1826394005916816 max depth = 3
# 0.1768805385579121 -central air, 0.17248604946185436 - bdroom, 0.169248976411171 garage car, 0.15898297857213403 overallcond


0.15898297857213403
